In [8]:
from diffusers import AudioLDM2Pipeline
import torch


model_id = "cvssp/audioldm2"
pipe = AudioLDM2Pipeline.from_pretrained(model_id, torch_dtype=torch.float16)

pipe.to("cuda")
generator = torch.Generator("cuda").manual_seed(0)

Loading pipeline components...: 100%|██████████| 11/11 [00:03<00:00,  3.36it/s]


In [5]:
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

In [7]:
prompt = "The sound of Brazilian samba drums with waves gently crashing in the background"
audio = pipe(prompt, audio_length_in_s=15).audios[0]

  0%|          | 0/200 [00:00<?, ?it/s]


Unsupported: inline in skipfiles: Logger.info  | info /home/yihao/anaconda3/envs/audioldm/lib/python3.8/logging/__init__.py

from user code:
   File "/home/yihao/anaconda3/envs/audioldm/lib/python3.8/site-packages/diffusers/pipelines/audioldm2/modeling_audioldm2.py", line 720, in forward
    logger.info("Forward upsample size to force interpolation output size.")

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [4]:
negative_prompt = "Low quality, average quality."

audio = pipe(prompt, negative_prompt=negative_prompt, generator=generator.manual_seed(0)).audios[0]

100%|██████████| 200/200 [00:16<00:00, 12.38it/s]


In [3]:
from IPython.display import Audio
Audio(audio, rate=16000)


In [9]:
from diffusers import DPMSolverMultistepScheduler

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

audio = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=25,audio_length_in_s= 15).audios[0]

100%|██████████| 25/25 [00:02<00:00, 11.58it/s]


In [10]:
Audio(audio, rate=16000)